In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
import random


In [ ]:

df = pd.read_csv("phoneprices.csv")

df

In [ ]:

x = df.iloc[:,:-1]

x_normalized = (x - x.min()) / (x.max() - x.min())

x_normalized

In [ ]:
y = pd.get_dummies(df.iloc[:,-1], prefix='y')

y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_normalized, y, test_size=0.3)

len_input = len(x_train.columns)
len_output = len(y_train.columns)

print("Len Input: {}".format(len_input))
print("Len Output: {}".format(len_output))

In [ ]:
def objective_function(x, y, x_validation, y_validation, params):
    model = MLPClassifier(
        random_state=1, 
        max_iter=1000,
        alpha=params['alpha'],
        learning_rate_init=params['learning_rate']
    )

    model.fit(x, y)

    predictions = model.predict(x_validation)

    return mean_squared_error(predictions, y_validation)

In [ ]:
grid_parameters = [
    { 'alpha': 0.002, 'learning_rate': 0.01 },
    { 'alpha': 0.003, 'learning_rate': 0.009 },
    { 'alpha': 0.004, 'learning_rate': 0.008 },
    { 'alpha': 0.005, 'learning_rate': 0.007 },
    { 'alpha': 0.006, 'learning_rate': 0.006 },
    { 'alpha': 0.007, 'learning_rate': 0.005 },
    { 'alpha': 0.008, 'learning_rate': 0.004 },
    { 'alpha': 0.009, 'learning_rate': 0.003 }
]

In [ ]:
best_params = None
current_loss = 1000

for params in grid_parameters:
    loss = objective_function(x_train.values, y_train.values, x_test.values, y_test.values, params)
    print("Loss: {}".format(loss))

    if loss < current_loss:
        best_params = params

print("Best params: {}".format(best_params))

In [ ]:
num_samples = 10
best_params = None
current_loss = 1000

for i in range(num_samples):
    params = {
        'alpha': random.randrange(0, 100) * 0.0001,
        'learning_rate': random.randrange(0, 100) * 0.0001
    }

    loss = objective_function(x_train.values, y_train.values, x_test.values, y_test.values, params)
    print("Loss: {}".format(loss))

    if loss < current_loss:
        best_params = params

print("Best params: {}".format(best_params))

In [ ]:
x_samples = []
y_samples = []

num_samples = 10
for i in range(num_samples):
    params = {
        'alpha': random.randrange(0, 100) * 0.0001,
        'learning_rate': random.randrange(0, 100) * 0.0001
    }

    loss = objective_function(x_train.values, y_train.values, x_test.values, y_test.values, params)
    print("Loss: {}".format(loss))

    x1 = params['alpha']
    x2 = params['learning_rate']
    y = loss

    x_samples.append([x1, x2])
    y_samples.append([y])

In [ ]:
x_samples

In [ ]:
y_samples

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(x_samples, y_samples)

mean, std = gaussian_process.predict(x_samples, return_std=True)

std